In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tensorflow-great-barrier-reef/example_sample_submission.csv
/kaggle/input/tensorflow-great-barrier-reef/example_test.npy
/kaggle/input/tensorflow-great-barrier-reef/train.csv
/kaggle/input/tensorflow-great-barrier-reef/test.csv
/kaggle/input/tensorflow-great-barrier-reef/greatbarrierreef/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/tensorflow-great-barrier-reef/greatbarrierreef/__init__.py
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/6234.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/1269.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/3863.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/6241.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/10304.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/623.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video_1/2193.jpg
/kaggle/input/tensorflow-great-barrier-reef/train_images/video

In [2]:
import contextlib2
import io
import numpy as np
import IPython
import json
import os
import pathlib
import pandas as pd
import sys
import tensorflow as tf
import time

from PIL import Image, ImageDraw

# -- Importing the library which is used to submit the prediction result --
INPUT_DIR = '/kaggle/input/tensorflow-great-barrier-reef/'
sys.path.insert(0, INPUT_DIR)

# Setting up folders

In [3]:
# http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz
# http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
# http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz

CUSTOM_MODEL_NAME = 'my_efficientdet_d0' 
PRETRAINED_MODEL_NAME = 'efficientdet_d0_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [4]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'ANNOTATION_PATH_TRAIN': os.path.join('Tensorflow', 'workspace','annotations','train'),
    'ANNOTATION_PATH_TEST': os.path.join('Tensorflow', 'workspace','annotations','test'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [5]:
paths

{'WORKSPACE_PATH': 'Tensorflow/workspace',
 'SCRIPTS_PATH': 'Tensorflow/scripts',
 'APIMODEL_PATH': 'Tensorflow/models',
 'ANNOTATION_PATH': 'Tensorflow/workspace/annotations',
 'ANNOTATION_PATH_TRAIN': 'Tensorflow/workspace/annotations/train',
 'ANNOTATION_PATH_TEST': 'Tensorflow/workspace/annotations/test',
 'IMAGE_PATH': 'Tensorflow/workspace/images',
 'MODEL_PATH': 'Tensorflow/workspace/models',
 'PRETRAINED_MODEL_PATH': 'Tensorflow/workspace/pre-trained-models',
 'CHECKPOINT_PATH': 'Tensorflow/workspace/models/my_efficientdet_d0',
 'OUTPUT_PATH': 'Tensorflow/workspace/models/my_efficientdet_d0/export',
 'TFJS_PATH': 'Tensorflow/workspace/models/my_efficientdet_d0/tfjsexport',
 'TFLITE_PATH': 'Tensorflow/workspace/models/my_efficientdet_d0/tfliteexport',
 'PROTOC_PATH': 'Tensorflow/protoc'}

In [6]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [7]:
files

{'PIPELINE_CONFIG': 'Tensorflow/workspace/models/my_efficientdet_d0/pipeline.config',
 'TF_RECORD_SCRIPT': 'Tensorflow/scripts/generate_tfrecord.py',
 'LABELMAP': 'Tensorflow/workspace/annotations/label_map.pbtxt'}

In [8]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [9]:
!ls Tensorflow

models	protoc	scripts  workspace


# Downloading TF model for extra $$

In [10]:
if os.name=='nt':
    !pip install wget
    import wget

In [11]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 68979, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 68979 (delta 9), reused 14 (delta 8), pack-reused 68962
Receiving objects: 100% (68979/68979), 577.24 MiB | 34.36 MiB/s, done.
Resolving deltas: 100% (48624/48624), done.


In [12]:
%%bash
cd Tensorflow/models/research
wget https://storage.googleapis.com/odml-dataset/others/setup.py
protoc object_detection/protos/*.proto --python_out=.
pip install -q --user .
pip install .

Processing /kaggle/working/Tensorflow/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=21908846 sha256=06674aecc65dea01b1576249f0fadf0aa553fef4978ba7d56db2ca1fd0c70cd3
  Stored in directory: /tmp/pip-ephem-wheel-cache-66fb2mhj/wheels/34/5c/22/e3bd46089424ecec281196e3dee48b96c7fd46c5ae2d13f350
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


--2022-02-14 12:02:49--  https://storage.googleapis.com/odml-dataset/others/setup.py
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.212.128, 172.253.123.128, 172.217.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1401 (1.4K) [text/x-python]
Saving to: ‘setup.py’

     0K .                                                     100% 26.5M=0s

2022-02-14 12:02:50 (26.5 MB/s) - ‘setup.py’ saved [1401/1401]

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-l

In [13]:
# Test if the Object Dectection API is working correctly
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2022-02-14 12:04:32.821930: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 12:04:32.886854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-02-14 12:04:32.896529: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping

# Some Files

In [14]:
%%writefile ./Tensorflow/scripts/cross_validation_setup.py
"""
Splits a dataframe to a specified cross_validation framework
"""

import pandas as pd
import numpy as np
import argparse
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

parser = argparse.ArgumentParser(
    description="Cross Validation Setup")
parser.add_argument("-in",
                    "--input_dir",
                    help="The filepath to the input csv.", type=str)
parser.add_argument("-out",
                    "--output_dir",
                    help="The filepath to the output csv.", type=str)
parser.add_argument("-type",
                    "--type",
                    help="The type of cross_validation to use.", type=str)
parser.add_argument("-on",
                    "--on",
                    help="The column to stratisfy on.", type=str)
parser.add_argument("-folds",
                    "--folds",
                    help="The number of folds to create.", type=int)
parser.add_argument("-hold",
                    "--holdout",
                    help="The fold to holdout.", type=int)

args = parser.parse_args()

def cross_validation():
    df = pd.read_csv(args.input_dir)
    df = df[df.annotations!='[]']
    df = df.reset_index(drop=True)
    
    if args.type=='kfold':
        kf = KFold(n_splits=args.folds, shuffle=True, random_state=42)
        for f, (t_, v_) in enumerate(kf.split(X=df)):
            df.loc[v_, 'fold'] = f
            
    elif args.type=='skfold' and args.on=='video_id':
        # uses skf to get even distrubution of data from different videos
        kf = StratifiedKFold(n_splits=args.folds, shuffle=True, random_state=42)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.video_id)): 
            df.loc[v_, 'fold'] = f
            
    elif args.type=='gkfold' and args.on=='sequence':
        kf = GroupKFold(n_splits=args.folds)
        for f, (t_, v_) in enumerate(kf.split(X=df, y=df.video_id, groups=df.sequence)): 
            df.loc[v_, 'fold'] = f
            
    else:
        raise Exception('Not Implemented')
        
    df.to_csv(args.output_dir, index=False)
    
if __name__ == '__main__':
    cross_validation()

Writing ./Tensorflow/scripts/cross_validation_setup.py


In [15]:
%%writefile ./Tensorflow/scripts/generate_tfrecords.py

import os
from os.path import exists
import glob
import pandas as pd
import io
import json
import xml.etree.ElementTree as ET
import contextlib2
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from object_detection.dataset_tools import tf_record_creation_util
from collections import namedtuple

# Initiate argument parser
parser = argparse.ArgumentParser(
    description="TensorFlow TFRecord Generator")
parser.add_argument("-c",
                    "--csv_path",
                    help="Path to the train.csv file.", type=str)
parser.add_argument("-o",
                    "--output_path",
                    help="Path of output TFRecord (.record) file.", type=str)
parser.add_argument("-i",
                    "--image_dir",
                    help="Path to the folder where the input image files are stored.", type=str)
parser.add_argument("-t",
                    "--train",
                    help="True if this is a training dataset, false if it is a validation dataset.", type=str)
parser.add_argument("-s",
                    "--shards",
                    help="The number of shards for the dataset", type=int)
parser.add_argument("-f",
                    "--holdout_fold",
                    help="The fold to holdout.", type=int)

args = parser.parse_args()

def create_tf_example(data_df: pd.DataFrame, video_id: int, video_frame: int):
    """
    Create a tf.Example entry for a given training image.
    """
    full_path = os.path.join(args.image_dir, os.path.join(f'video_{video_id}', f'{video_frame}.jpg'))
    with tf.io.gfile.GFile(full_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')

    height = image.size[1] # Image height
    width = image.size[0] # Image width
    filename = f'{video_id}:{video_frame}'.encode('utf8') # Unique id of the image.
    encoded_image_data = None # Encoded image bytes
    image_format = 'jpeg'.encode('utf8') # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    rows = data_df[(data_df.video_id == video_id) & (data_df.video_frame == video_frame)]
    for _, row in rows.iterrows():
        annotations = json.loads(row.annotations.replace("'", '"'))
        for annotation in annotations:
            xmins.append(annotation['x'] / width) 
            xmaxs.append((annotation['x'] + annotation['width']) / width) 
            ymins.append(annotation['y'] / height) 
            ymaxs.append((annotation['y'] + annotation['height']) / height) 

            classes_text.append('COTS'.encode('utf8'))
            classes.append(1)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example

def create_labels_file():
    label_map_str = """
    item {
        id: 1
        name: 'COTS'
        }
                    """

    if exists('dataset/label_map.pbtxt') is False:
        with open('dataset/label_map.pbtxt', 'w') as f:
            f.write(label_map_str)
        print('Successfully created label_map.pbtxt file')

if __name__ == '__main__':

    # label file
    create_labels_file()
    #writer = tf.python_io.TFRecordWriter(args.output_path)
    
    # setup df
    data_df = pd.read_csv(args.csv_path)
    if args.train =='train':
        data_df = data_df[data_df.fold != args.holdout_fold].reset_index(drop=True)
    else:
        data_df = data_df[data_df.fold == args.holdout_fold].reset_index(drop=True)
    
    # make records
    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
            tf_record_close_stack, args.output_path, args.shards)
        
        for index, row in data_df.iterrows():
            tf_example = create_tf_example(data_df, row.video_id, row.video_frame)
            output_shard_index = index % args.shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
    #writer.close()
    print('Successfully created the TFRecord file: {}'.format(args.output_path))

Writing ./Tensorflow/scripts/generate_tfrecords.py


In [16]:
%%bash

python ./Tensorflow/scripts/cross_validation_setup.py \
    -in ../input/tensorflow-great-barrier-reef/train.csv \
    -out train.csv \
    -type gkfold \
    -on sequence \
    -folds 10 \
    -hold 0

mkdir dataset

python ./Tensorflow/scripts/generate_tfrecords.py \
    -c train.csv \
    -o Tensorflow/workspace/annotations/train \
    -i ../input/tensorflow-great-barrier-reef/train_images \
    -t train \
    -s 4 \
    -f 0

python ./Tensorflow/scripts/generate_tfrecords.py \
    -c train.csv \
    -o Tensorflow/workspace/annotations/test \
    -i ../input/tensorflow-great-barrier-reef/train_images \
    -t valid \
    -s 4 \
    -f 0

Successfully created label_map.pbtxt file
Successfully created the TFRecord file: Tensorflow/workspace/annotations/train
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test


In [17]:
!ls Tensorflow/workspace/annotations/

test		     test-00003-of-00004   train-00002-of-00004
test-00000-of-00004  train		   train-00003-of-00004
test-00001-of-00004  train-00000-of-00004
test-00002-of-00004  train-00001-of-00004


In [18]:
import object_detection
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2022-02-14 12:06:15--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.203.128, 2607:f8b0:400c:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.203.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

efficientdet_d0_coc 100%[===================>]  29.31M   172MB/s    in 0.2s    

2022-02-14 12:06:15 (172 MB/s) - ‘efficientdet_d0_coco17_tpu-32.tar.gz’ saved [30736482/30736482]

efficientdet_d0_coco17_tpu-32/
efficientdet_d0_coco17_tpu-32/checkpoint/
efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d0_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0.index
efficientdet_d0_coco17_tpu-32/pipeline.config
efficientdet_d0_coco17_tpu-32/saved_model/
efficientdet

In [19]:
"""
%%bash
cd Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_320x320_coco17_tpu-8

ls
"""

'\n%%bash\ncd Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_320x320_coco17_tpu-8\n\nls\n'

# Config File

In [20]:
!ls Tensorflow/workspace/pre-trained-models/efficientdet_d0_coco17_tpu-32

checkpoint  pipeline.config  saved_model


In [21]:
MODEL_NAME = "efficientdet_d0_coco17_tpu-32"
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

In [22]:
%%bash

cd Tensorflow/workspace/pre-trained-models/efficientdet_d0_coco17_tpu-32/checkpoint
ls

checkpoint
ckpt-0.data-00000-of-00001
ckpt-0.index


# Update config

In [23]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [24]:
!ls Tensorflow/workspace/models/my_efficientdet_d0

export	pipeline.config  tfjsexport  tfliteexport


In [25]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
#config = config_util.get_configs_from_pipeline_file(files['./Tensorflow/workspace/models/my_ssd_mobilenet_320/'])

In [26]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 512
       max_dimension: 512
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "ssd_efficientnet-b0_bifpn_keras"
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: SWISH
       batch_norm {
         decay: 0.9900000095367432
         scale: true
         epsilon: 0.0010000000474974513
       }
       force_use_bias: true
     }
     bifpn {
       min_level: 3
       max_level: 7
       num_iterations: 3
       num_filters: 64
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 1.0
       x_scale: 1.0
       height_scale: 1.0
       width_scale: 1.0
     }
   }
   matcher {
     argmax_matc

In [27]:
!python tensorflow --version

python: can't open file 'tensorflow': [Errno 2] No such file or directory


In [28]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [29]:
# * -- Investigate This -- * #

labels = [{'id':1, 'name':'COTS'}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [30]:
len(labels)

1

In [31]:
pipeline_config.train_config.batch_size

128

In [32]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
#pipeline_config.train_input_reader.label_map_path= os.path.join(paths['./dataset/label_map.pbtxt'])
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train-?????-of-00004')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
#pipeline_config.eval_input_reader[0].label_map_path = files['./dataset/label_map.pbtxt']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test-?????-of-00004')]

In [33]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [34]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 512
       max_dimension: 512
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "ssd_efficientnet-b0_bifpn_keras"
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: SWISH
       batch_norm {
         decay: 0.9900000095367432
         scale: true
         epsilon: 0.0010000000474974513
       }
       force_use_bias: true
     }
     bifpn {
       min_level: 3
       max_level: 7
       num_iterations: 3
       num_filters: 64
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 1.0
       x_scale: 1.0
       height_scale: 1.0
       width_scale: 1.0
     }
   }
   matcher {
     argmax_matc

# Train

In [35]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [36]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [37]:
command

'python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_efficientdet_d0 --pipeline_config_path=Tensorflow/workspace/models/my_efficientdet_d0/pipeline.config --num_train_steps=2000'

In [38]:
!ls Tensorflow/workspace/annotations/

label_map.pbtxt      test-00002-of-00004   train-00001-of-00004
test		     test-00003-of-00004   train-00002-of-00004
test-00000-of-00004  train		   train-00003-of-00004
test-00001-of-00004  train-00000-of-00004


In [39]:
!{command}

2022-02-14 12:06:32.617091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 12:06:32.623280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-02-14 12:06:32.624290: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and s

# Eval

In [40]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [41]:
command

'python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_efficientdet_d0 --pipeline_config_path=Tensorflow/workspace/models/my_efficientdet_d0/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_efficientdet_d0'

In [42]:
!ls Tensorflow/workspace/models/my_efficientdet_d0

checkpoint		    ckpt-2.index		pipeline.config
ckpt-1.data-00000-of-00001  ckpt-3.data-00000-of-00001	tfjsexport
ckpt-1.index		    ckpt-3.index		tfliteexport
ckpt-2.data-00000-of-00001  export			train


In [43]:
!{command}

2022-02-14 15:00:54.940486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 15:00:54.957065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-02-14 15:00:54.959006: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and s

# Export

In [44]:
!mkdir Tensorflow/workspace/models/my_efficientdet_d0/output

In [45]:
!python Tensorflow/models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path={Tensorflow/workspace/models/my_efficientdet_d0} \
    --trained_checkpoint_dir={Tensorflow/workspace/models/my_efficientdet_d0} \
    --output_directory={Tensorflow/workspace/models/my_efficientdet_d0/output}

2022-02-14 16:08:30.873284: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-14 16:08:30.879337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-02-14 16:08:30.880348: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping

In [46]:
!ls {Tensorflow/workspace/models/my_efficientdet_d0}/output

ls: cannot access '{Tensorflow/workspace/models/my_efficientdet_d0}/output': No such file or directory


In [47]:
!ls

Tensorflow  __notebook__.ipynb	dataset  train.csv
